<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/StructureDistance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install --force-reinstall 'sqlalchemy<2.0.0'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.7/613.7 kB 56.3 MB/s eta 0:00:00
  Attempting uninstall: greenlet
    Found existing installation: greenlet 2.0.2
    Uninstalling greenlet-2.0.2:
      Successfully uninstalled greenlet-2.0.2
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.19
    Uninstalling SQLAlchemy-2.0.19:
      Successfully uninstalled SQLAlchemy-2.0.19
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.


In [ ]:
import os
!pwd
os.chdir('/content')
# Clone ALIGNN repo to get example folder
if not os.path.exists('graphdot'):
  !git clone https://gitlab.com/knc6/graphdot.git

os.chdir('graphdot')
# Install using setup.py in case pip didn't work
# !python setup.py devel!op
!pip install -e .
#!pip install dgl-cu111 # Colab has cuda 11.1

/content
Cloning into 'graphdot'...
remote: Enumerating objects: 8001, done.
remote: Counting objects: 100% (3928/3928), done.
remote: Compressing objects: 100% (1259/1259), done.
remote: Total 8001 (delta 2613), reused 3923 (delta 2611), pack-reused 4073
Receiving objects: 100% (8001/8001), 3.08 MiB | 2.87 MiB/s, done.
Resolving deltas: 100% (5078/5078), done.
Obtaining file:///content/graphdot
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 90.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.1/353.1 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.8/865.8 kB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
pip install ase

In [ ]:
pip install pymatgen

In [ ]:
import numpy as np
import pandas as pd
from ase.build import molecule, bulk
from graphdot import Graph
from graphdot.kernel.molecular import Tang2019MolecularKernel

# build sample molecules
small_title = ['H2O', 'HCl', 'NaCl']
bulk_title = ['NaCl-bulk', 'NaCl-bulk2']
bulk = [
    bulk('NaCl', 'rocksalt', a=5.64),
    bulk('NaCl', 'rocksalt', a=5.66),
]
molecules = [molecule(name) for name in small_title] + bulk

# convert to molecular graphs
graphs = [Graph.from_ase(m) for m in molecules]

# use pre-defined molecular kernel
kernel = Tang2019MolecularKernel(edge_length_scale=0.1)

R = kernel(graphs)

# normalize the similarity matrix
d = np.diag(R)**-0.5
K = np.diag(d).dot(R).dot(np.diag(d))

# note the difference between the NaCl variants
title = small_title + bulk_title
print(pd.DataFrame(K, columns=title, index=title))

In [ ]:
pip install alignn jarvis-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 15.1 MB/s eta 0:00:00


In [ ]:

from jarvis.db.figshare import data
import pandas as pd
from jarvis.core.atoms import Atoms

dft_3d = data('dft_3d')
df=pd.DataFrame(dft_3d)

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699


100%|██████████| 40.8M/40.8M [00:03<00:00, 10.6MiB/s]


Loading the zipfile...
Loading completed.


In [ ]:
si=['JVASP-1002','JVASP-91933','JVASP-25369']#,'JVASP-25368']
mos2=['JVASP-28733','JVASP-28413','JVASP-58505']
ni3al=['JVASP-14971','JVASP-99749','JVASP-11979']
sio2=['JVASP-58349', 'JVASP-34674','JVASP-34656']

In [ ]:
def atoms_jid(ids=[],verbose=True):
  get_atoms=[]
  for i in ids:
    atoms=(Atoms.from_dict(df[df['jid']==i].atoms.values[0]))
    if verbose:
        print(i,atoms.composition.reduced_formula,atoms.get_spacegroup)
    ase_atoms=atoms.ase_converter()
    get_atoms.append(ase_atoms)
  return get_atoms

def get_df(ids=[],verbose=True):
    ase_atoms_list = atoms_jid(ids,verbose=verbose)
    graphs = []
    ids_list = []
    for i,j in zip (ids,ase_atoms_list):
      try:
          graphs.append(Graph.from_ase(j) )
          #ids_list.append(i)
          ids_list.append(j.get_chemical_formula())
      except Exception as exp:
        print('exp',exp,i)
        pass

    # graphs = [Graph.from_ase(m) for m in ase_atoms_list]

    # use pre-defined molecular kernel
    kernel = Tang2019MolecularKernel(edge_length_scale=0.1)

    R = kernel(graphs)

    # normalize the similarity matrix
    d = np.diag(R)**-0.5
    K = np.diag(d).dot(R).dot(np.diag(d))

    # note the difference between the NaCl variants

    df_kernel = (pd.DataFrame(K, columns=ids_list, index=ids_list))
    return df_kernel

# si_atoms = atoms_jid(si)

In [ ]:
get_df(si)

JVASP-1002 Si [227, 'Fd-3m']
JVASP-91933 Si [12, 'C2/m']
JVASP-25369 Si [223, 'Pm-3n']


,Si2,Si8,Si46
Si2,1.000000,0.233288,0.163287
Si8,0.233288,1.000000,0.869847
Si46,0.163287,0.869847,1.000000


In [ ]:
get_df(ni3al)

JVASP-14971 AlNi3 [221, 'Pm-3m']
JVASP-99749 AlNi3 [194, 'P6_3/mmc']
JVASP-11979 AlNi3 [225, 'Fm-3m']


,AlNi3,Al2Ni6,AlNi3
AlNi3,1.000000,0.954035,0.711243
Al2Ni6,0.954035,1.000000,0.808739
AlNi3,0.711243,0.808739,1.000000


In [ ]:
get_df(mos2)

JVASP-28733 MoS2 [194, 'P6_3/mmc']
JVASP-28413 MoS2 [164, 'P-3m1']
JVASP-58505 MoS2 [216, 'F-43m']


,Mo4S8,Mo2S4,Mo4S8
Mo4S8,1.000000,0.973766,0.881506
Mo2S4,0.973766,1.000000,0.857583
Mo4S8,0.881506,0.857583,1.000000


In [ ]:
get_df(sio2)

JVASP-58349 SiO2 [152, 'P3_121']
JVASP-34674 SiO2 [20, 'C222_1']
JVASP-34656 SiO2 [92, 'P4_12_12']


,O6Si3,O8Si4,O16Si8
O6Si3,1.000000,0.981050,0.930632
O8Si4,0.981050,1.000000,0.963703
O16Si8,0.930632,0.963703,1.000000


In [ ]:
x = get_df(si+sio2+ni3al+mos2)

JVASP-1002 Si [227, 'Fd-3m']
JVASP-91933 Si [12, 'C2/m']
JVASP-25369 Si [223, 'Pm-3n']
JVASP-58349 SiO2 [152, 'P3_121']
JVASP-34674 SiO2 [20, 'C222_1']
JVASP-34656 SiO2 [92, 'P4_12_12']
JVASP-14971 AlNi3 [221, 'Pm-3m']
JVASP-99749 AlNi3 [194, 'P6_3/mmc']
JVASP-11979 AlNi3 [225, 'Fm-3m']
JVASP-28733 MoS2 [194, 'P6_3/mmc']
JVASP-28413 MoS2 [164, 'P-3m1']
JVASP-58505 MoS2 [216, 'F-43m']


In [ ]:
x

,Si2,Si8,Si46,O6Si3,O8Si4,O16Si8,AlNi3,Al2Ni6,AlNi3,Mo4S8,Mo2S4,Mo4S8
Si2,1.000000,0.233288,0.163287,0.063762,0.068309,0.073951,0.020443,0.024192,0.029055,0.031724,0.029984,0.032827
Si8,0.233288,1.000000,0.869847,0.372574,0.399232,0.436911,0.115175,0.136780,0.160568,0.176673,0.164690,0.185581
Si46,0.163287,0.869847,1.000000,0.390146,0.422565,0.459150,0.118733,0.140791,0.164615,0.181053,0.168165,0.191868
O6Si3,0.063762,0.372574,0.390146,1.000000,0.981050,0.930632,0.133847,0.158554,0.181572,0.199133,0.184058,0.213564
O8Si4,0.068309,0.399232,0.422565,0.981050,1.000000,0.963703,0.143302,0.169617,0.194363,0.213075,0.197096,0.229485
O16Si8,0.073951,0.436911,0.459150,0.930632,0.963703,1.000000,0.154231,0.182902,0.209979,0.231148,0.213520,0.247891
AlNi3,0.020443,0.115175,0.118733,0.133847,0.143302,0.154231,1.000000,0.954035,0.711243,0.146091,0.136758,0.156315
Al2Ni6,0.024192,0.136780,0.140791,0.158554,0.169617,0.182902,0.954035,1.000000,0.808739,0.173126,0.161741,0.184594
AlNi3,0.029055,0.160568,0.164615,0.181572,0.194363,0.209979,0.711243,0.808739,1.000000,0.202329,0.189739,0.213131
Mo4S8,0.031724,0.176673,0.181053,0.199133,0.213075,0.231148,0.146091,0.173126,0.202329,1.000000,0.973766,0.881506


In [ ]:
import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)
x=x.style.background_gradient(cmap=cm)
display(x)

KeyError: ignored

In [ ]:
%%time
todo_jids = df.jid.values[0:10]
x = get_df(todo_jids)
import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)
x=x.style.background_gradient(cmap=cm)
display(x)

JVASP-90856 TiCuSiAs [129, 'P4/nmm']
JVASP-86097 DyB6 [221, 'Pm-3m']
JVASP-64906 Be2OsRu [119, 'I-4m2']
JVASP-98225 KBi [14, 'P2_1/c']
JVASP-10 VSe2 [164, 'P-3m1']
JVASP-14014 TbMnSi [62, 'Pnma']
JVASP-64664 Ba4NaBi [216, 'F-43m']
JVASP-22556 SrFeO3 [221, 'Pm-3m']
JVASP-86726 LuNi4Sn [216, 'F-43m']
JVASP-28634 MoW3Se2S6 [156, 'P3m1']


,As2Cu2Si2Ti2,B6Dy,Be2OsRu,Bi16K16,Se2V,Mn4Si4Tb4,Ba4BiNa,FeO3Sr,LuNi4Sn,MoS6Se2W3
As2Cu2Si2Ti2,1.000000,0.189377,0.311897,0.338227,0.220507,0.512672,0.257625,0.303368,0.272474,0.355973
B6Dy,0.189377,1.000000,0.139857,0.164077,0.102118,0.185049,0.124559,0.142435,0.131404,0.168204
Be2OsRu,0.311897,0.139857,1.000000,0.249179,0.177104,0.284402,0.189746,0.225071,0.209017,0.273197
Bi16K16,0.338227,0.164077,0.249179,1.000000,0.184670,0.326297,0.304807,0.250451,0.230247,0.301540
Se2V,0.220507,0.102118,0.177104,0.184670,1.000000,0.207347,0.141604,0.158528,0.152740,0.298180
Mn4Si4Tb4,0.512672,0.185049,0.284402,0.326297,0.207347,1.000000,0.248343,0.281141,0.267901,0.339510
Ba4BiNa,0.257625,0.124559,0.189746,0.304807,0.141604,0.248343,1.000000,0.190674,0.174877,0.229343
FeO3Sr,0.303368,0.142435,0.225071,0.250451,0.158528,0.281141,0.190674,1.000000,0.199347,0.258509
LuNi4Sn,0.272474,0.131404,0.209017,0.230247,0.152740,0.267901,0.174877,0.199347,1.000000,0.246001
MoS6Se2W3,0.355973,0.168204,0.273197,0.301540,0.298180,0.339510,0.229343,0.258509,0.246001,1.000000


CPU times: user 416 ms, sys: 251 ms, total: 667 ms
Wall time: 500 ms


In [ ]:
%%time
todo_jids = df.jid.values[0:100]
x = get_df(todo_jids)
import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)
x=x.style.background_gradient(cmap=cm)
display(x)

JVASP-90856 TiCuSiAs [129, 'P4/nmm']
JVASP-86097 DyB6 [221, 'Pm-3m']
JVASP-64906 Be2OsRu [119, 'I-4m2']
JVASP-98225 KBi [14, 'P2_1/c']
JVASP-10 VSe2 [164, 'P-3m1']
JVASP-14014 TbMnSi [62, 'Pnma']
JVASP-64664 Ba4NaBi [216, 'F-43m']
JVASP-22556 SrFeO3 [221, 'Pm-3m']
JVASP-86726 LuNi4Sn [216, 'F-43m']
JVASP-28634 MoW3Se2S6 [156, 'P3m1']
JVASP-98550 Ba4CeMn3O12 [166, 'R-3m']
JVASP-89265 Fe2SiO4 [62, 'Pnma']
JVASP-97915 Ba14Na14LiN6 [225, 'Fm-3m']
JVASP-90228 Ag2IO6 [148, 'R-3']
JVASP-42300 Li4Mn3Cr3O12 [22, 'F222']
JVASP-86968 CdAs2O6 [162, 'P-31m']
JVASP-90532 FeOF [146, 'R3']
JVASP-50332 Na2CuO2 [71, 'Immm']
JVASP-86205 SmSb12Os4 [204, 'Im-3']
JVASP-28704 Mo3WSe6S2 [156, 'P3m1']
JVASP-60702 PdSe8Cl2 [61, 'Pbca']
JVASP-97799 BaGe2 [62, 'Pnma']
JVASP-28565 Te2Mo2WSe2S2 [156, 'P3m1']
JVASP-86436 LiSbF6 [148, 'R-3']
JVASP-1372 AlAs [216, 'F-43m']
JVASP-14873 SrB6 [221, 'Pm-3m']
JVASP-28397 SiS [53, 'Pmna']
JVASP-98284 Ce2Al16Pt9 [71, 'Immm']
JVASP-89025 Na2Si2Se5 [63, 'Cmcm']
JVASP-1996 NaI 

CPU times: user 4.95 s, sys: 2.05 s, total: 7 s
Wall time: 5.04 s


In [ ]:
%%time
todo_jids = df.jid.values[0:1000]
x = get_df(todo_jids)
x
# import seaborn as sns
# cm = sns.light_palette("blue", as_cmap=True)
# x=x.style.background_gradient(cmap=cm)
# display(x)

JVASP-90856 TiCuSiAs [129, 'P4/nmm']
JVASP-86097 DyB6 [221, 'Pm-3m']
JVASP-64906 Be2OsRu [119, 'I-4m2']
JVASP-98225 KBi [14, 'P2_1/c']
JVASP-10 VSe2 [164, 'P-3m1']
JVASP-14014 TbMnSi [62, 'Pnma']
JVASP-64664 Ba4NaBi [216, 'F-43m']
JVASP-22556 SrFeO3 [221, 'Pm-3m']
JVASP-86726 LuNi4Sn [216, 'F-43m']
JVASP-28634 MoW3Se2S6 [156, 'P3m1']
JVASP-98550 Ba4CeMn3O12 [166, 'R-3m']
JVASP-89265 Fe2SiO4 [62, 'Pnma']
JVASP-97915 Ba14Na14LiN6 [225, 'Fm-3m']
JVASP-90228 Ag2IO6 [148, 'R-3']
JVASP-42300 Li4Mn3Cr3O12 [22, 'F222']
JVASP-86968 CdAs2O6 [162, 'P-31m']
JVASP-90532 FeOF [146, 'R3']
JVASP-50332 Na2CuO2 [71, 'Immm']
JVASP-86205 SmSb12Os4 [204, 'Im-3']
JVASP-28704 Mo3WSe6S2 [156, 'P3m1']
JVASP-60702 PdSe8Cl2 [61, 'Pbca']
JVASP-97799 BaGe2 [62, 'Pnma']
JVASP-28565 Te2Mo2WSe2S2 [156, 'P3m1']
JVASP-86436 LiSbF6 [148, 'R-3']
JVASP-1372 AlAs [216, 'F-43m']
JVASP-14873 SrB6 [221, 'Pm-3m']
JVASP-28397 SiS [53, 'Pmna']
JVASP-98284 Ce2Al16Pt9 [71, 'Immm']
JVASP-89025 Na2Si2Se5 [63, 'Cmcm']
JVASP-1996 NaI 

,As2Cu2Si2Ti2,B6Dy,Be2OsRu,Bi16K16,Se2V,Mn4Si4Tb4,Ba4BiNa,FeO3Sr,LuNi4Sn,MoS6Se2W3,...,Li4Ni4O16P4,CrO3Sr,Na6P2,Fe16Si7Ta6,Pb4S4,Ba4CrV,F2Mn6O10,BaInY,Ba4MgPt,Ba4SiTi
As2Cu2Si2Ti2,1.000000,0.189377,0.311897,0.338227,0.220507,0.512672,0.257625,0.303368,0.272474,0.355973,...,0.379398,0.301893,0.296761,0.456985,0.338180,0.257774,0.350099,0.305334,0.257350,0.346200
B6Dy,0.189377,1.000000,0.139857,0.164077,0.102118,0.185049,0.124559,0.142435,0.131404,0.168204,...,0.183314,0.141950,0.143393,0.174406,0.162720,0.124425,0.169142,0.144056,0.123154,0.124898
Be2OsRu,0.311897,0.139857,1.000000,0.249179,0.177104,0.284402,0.189746,0.225071,0.209017,0.273197,...,0.280929,0.221414,0.215489,0.274965,0.248760,0.189541,0.258633,0.219414,0.187605,0.190261
Bi16K16,0.338227,0.164077,0.249179,1.000000,0.184670,0.326297,0.304807,0.250451,0.230247,0.301540,...,0.324605,0.250500,0.253810,0.312770,0.289577,0.225585,0.298777,0.259294,0.221290,0.224713
Se2V,0.220507,0.102118,0.177104,0.184670,1.000000,0.207347,0.141604,0.158528,0.152740,0.298180,...,0.204258,0.157563,0.157288,0.200311,0.179855,0.170934,0.187938,0.160516,0.137318,0.139476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ba4CrV,0.257774,0.124425,0.189541,0.225585,0.170934,0.249111,0.676169,0.190523,0.174688,0.228561,...,0.247636,0.216065,0.193673,0.237269,0.219363,1.000000,0.228316,0.377576,0.625614,0.687199
F2Mn6O10,0.350099,0.169142,0.258633,0.298777,0.187938,0.485786,0.228084,0.677793,0.238764,0.307050,...,0.781287,0.683005,0.259608,0.320561,0.295822,0.228316,1.000000,0.264552,0.226027,0.229289
BaInY,0.305334,0.144056,0.219414,0.259294,0.160516,0.289256,0.370923,0.222988,0.202239,0.264698,...,0.289851,0.223404,0.228646,0.276412,0.256824,0.377576,0.264552,1.000000,0.411890,0.403869
Ba4MgPt,0.257350,0.123154,0.187605,0.221290,0.137318,0.247625,0.505993,0.189025,0.172905,0.225800,...,0.246138,0.189393,0.193006,0.235914,0.218139,0.625614,0.226027,0.411890,1.000000,0.808082


In [ ]:
%%time
todo_jids = df.jid.values[0:10000]
x = get_df(todo_jids)
import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)
x=x.style.background_gradient(cmap=cm)
display(x)

JVASP-90856 TiCuSiAs [129, 'P4/nmm']
JVASP-86097 DyB6 [221, 'Pm-3m']
JVASP-64906 Be2OsRu [119, 'I-4m2']
JVASP-98225 KBi [14, 'P2_1/c']
JVASP-10 VSe2 [164, 'P-3m1']
JVASP-14014 TbMnSi [62, 'Pnma']
JVASP-64664 Ba4NaBi [216, 'F-43m']
JVASP-22556 SrFeO3 [221, 'Pm-3m']
JVASP-86726 LuNi4Sn [216, 'F-43m']
JVASP-28634 MoW3Se2S6 [156, 'P3m1']
JVASP-98550 Ba4CeMn3O12 [166, 'R-3m']
JVASP-89265 Fe2SiO4 [62, 'Pnma']
JVASP-97915 Ba14Na14LiN6 [225, 'Fm-3m']
JVASP-90228 Ag2IO6 [148, 'R-3']
JVASP-42300 Li4Mn3Cr3O12 [22, 'F222']
JVASP-86968 CdAs2O6 [162, 'P-31m']
JVASP-90532 FeOF [146, 'R3']
JVASP-50332 Na2CuO2 [71, 'Immm']
JVASP-86205 SmSb12Os4 [204, 'Im-3']
JVASP-28704 Mo3WSe6S2 [156, 'P3m1']
JVASP-60702 PdSe8Cl2 [61, 'Pbca']
JVASP-97799 BaGe2 [62, 'Pnma']
JVASP-28565 Te2Mo2WSe2S2 [156, 'P3m1']
JVASP-86436 LiSbF6 [148, 'R-3']
JVASP-1372 AlAs [216, 'F-43m']
JVASP-14873 SrB6 [221, 'Pm-3m']
JVASP-28397 SiS [53, 'Pmna']
JVASP-98284 Ce2Al16Pt9 [71, 'Immm']
JVASP-89025 Na2Si2Se5 [63, 'Cmcm']
JVASP-1996 NaI 

In [ ]:
%%time
todo_jids = df.jid.values #[0:100]
x = get_df(todo_jids)
import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)
x=x.style.background_gradient(cmap=cm)
display(x)

In [ ]:
from jarvis.core.atoms import Atoms
from jarvis.core.graphs import Graph
from alignn.pretrained import get_figshare_model
model = get_figshare_model()
box = [[2.715, 2.715, 0], [0, 2.715, 2.715], [2.715, 0, 2.715]]
coords = [[0, 0, 0], [0.25, 0.25, 0.25]]
elements = ["Si", "Si"]
atoms = Atoms(lattice_mat=box, coords=coords, elements=elements)

def get_val(model,g,lg):
    activation = {}
    def getActivation(name):
        # the hook signature
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook
    h = model.readout.register_forward_hook(getActivation('readout'))
    out = model([g,lg])
    h.remove()
    return activation['readout'][0]


g,lg=Graph.atom_dgl_multigraph(atoms)
get_val(model,g,lg)